In [ ]:
# retrieve data from google analytics api
# reference: https://developers.google.com/analytics/devguides/reporting/core/v3/quickstart/service-py#pip
# query explorer, https://ga-dev-tools.google/query-explorer/


In [ ]:
# ! pip install google-api-python-client
# ! pip install google-auth-httplib2
# ! pip install google-auth-oauthlib

In [ ]:
from googleapiclient.discovery import build

In [ ]:
"""A simple example of how to access the Google Analytics API."""

# from googleapiclient.discovery import build
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

def get_service(api_name, api_version, scopes, key_file_location):
    """Get a service that communicates to a Google API.

    Args:
        api_name: The name of the api to connect to.
        api_version: The api version to connect to.
        scopes: A list auth scopes to authorize for the application.
        key_file_location: The path to a valid service account JSON key file.

    Returns:
        A service that is connected to the specified API.
    """

    credentials = ServiceAccountCredentials.from_json_keyfile_name(
            key_file_location, scopes=scopes)

    # Build the service object.
    service = build(api_name, api_version, credentials=credentials)

    return service


def get_first_profile_id(service):
    # Use the Analytics service object to get the first profile id.

    # Get a list of all Google Analytics accounts for this user
    accounts = service.management().accounts().list().execute()

    if accounts.get('items'):
        # Get the first Google Analytics account.
        account = accounts.get('items')[0].get('id')

        # Get a list of all the properties for the first account.
        properties = service.management().webproperties().list(
                accountId=account).execute()

        if properties.get('items'):
            # Get the first property id.
            property = properties.get('items')[0].get('id')

            # Get a list of all views (profiles) for the first property.
            profiles = service.management().profiles().list(
                    accountId=account,
                    webPropertyId=property).execute()

            if profiles.get('items'):
                # return the first view (profile) id.
                return profiles.get('items')[0].get('id')

    return None


def get_results(service, profile_id):
    # Use the Analytics Service Object to query the Core Reporting API
    # for the number of sessions within the past seven days.
    return service.data().ga().get(
            ids='ga:' + profile_id,
            start_date='2023-01-01',
            end_date='2023-01-06',
            metrics='ga:sessions').execute()


def print_results(results):
    # Print data nicely for the user.
    if results:
        print('View (Profile):', results.get('profileInfo').get('profileName'))
        print('Total Sessions:', results.get('rows')[0][0])

    else:
        print('No results found')


def main():
    # Define the auth scopes to request.
    # scope = 'https://www.googleapis.com/auth/analytics.readonly'
    key_file_location = './credentials/reports-service-account-google-analytics.json'#'<REPLACE_WITH_JSON_FILE>'

    # Authenticate and construct service.
    service = get_service(
            api_name='analytics',
            api_version='v3',
            scopes=[],
            key_file_location=key_file_location)

    # profile_id = get_first_profile_id(service)
    profile_id = "23837774"
    print_results(get_results(service, profile_id))

main()
